In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import io
import glob
import re
import json
import pickle
import string
from collections import Counter
import operator

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

In [3]:
#pre_process function that converts to lower case , removes punctuations and also does tokenization and lemmatization
def pre_process(s):
    s = s.lower()
    l=len(string.punctuation)
    s = s.translate(s.maketrans(string.punctuation,' '*l,''))
    #s = re.sub('[^A-Za-z\s\n ]+', ' ',s)
    
    t = word_tokenize(s)
    t = [lem.lemmatize(w) for w in t if w not in stopwords.words('english') and w.isalpha() and len(w)>1]
    return t

In [4]:
cwd = os.getcwd()
path = os.path.join(cwd, r"data_a2")
dirs = os.listdir(path)

In [5]:
#cls={0:"comp.graphics",1:"sci.med",2:"talk.politics.misc",3:"rec.sport.hockey",4:"sci.space"}
cls=['comp.graphics', 'rec.sport.hockey', 'sci.med', 'sci.space', 'talk.politics.misc']
docs={}
data ={'text':[],'label':[]}

In [17]:
x=[]
y=[]
yc=[]
for i in range(5):
    c=cls[i]
    files = os.listdir("data_a2/"+c)
    print(c,len(files))
    for j in range(len(files)):
        file="data_A2/"+c+"/"+files[j]
        with open(file, 'r', encoding="utf8", errors="ignore") as f:
            text = f.read()
            tl=pre_process(text)
            x.append(tl)
            y.append(i)
            yc.append(c)
    print(len(x))

comp.graphics 1000
1000
rec.sport.hockey 1000
2000
sci.med 1000
3000
sci.space 1000
4000
talk.politics.misc 1000
5000


In [18]:
data['text']=x
data['label']=yc
data = pd.DataFrame(data)
data

,text,label
0,"[xref, cantaloupe, srv, c, cmu, edu, comp, gra...",comp.graphics
1,"[path, cantaloupe, srv, c, cmu, edu, crabapple...",comp.graphics
2,"[path, cantaloupe, srv, c, cmu, edu, da, news,...",comp.graphics
3,"[xref, cantaloupe, srv, c, cmu, edu, comp, hum...",comp.graphics
4,"[path, cantaloupe, srv, c, cmu, edu, crabapple...",comp.graphics
...,...,...
4995,"[xref, cantaloupe, srv, c, cmu, edu, misc, leg...",talk.politics.misc
4996,"[xref, cantaloupe, srv, c, cmu, edu, talk, pol...",talk.politics.misc
4997,"[xref, cantaloupe, srv, c, cmu, edu, talk, pol...",talk.politics.misc
4998,"[path, cantaloupe, srv, c, cmu, edu, crabapple...",talk.politics.misc


In [47]:
data.to_csv("news_data.csv")

In [19]:
data['label'].unique()

array(['comp.graphics', 'rec.sport.hockey', 'sci.med', 'sci.space',
       'talk.politics.misc'], dtype=object)

In [3]:
data=pd.read_csv("news_data.csv")
data

,Unnamed: 0,text,label
0,0,"['xref', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",comp.graphics
1,1,"['path', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",comp.graphics
2,2,"['path', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",comp.graphics
3,3,"['xref', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",comp.graphics
4,4,"['path', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",comp.graphics
...,...,...,...
4995,4995,"['path', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",sci.space
4996,4996,"['path', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",sci.space
4997,4997,"['newsgroups', 'sci', 'space', 'path', 'cantal...",sci.space
4998,4998,"['xref', 'cantaloupe', 'srv', 'c', 'cmu', 'edu...",sci.space


In [20]:
labels=['comp.graphics', 'rec.sport.hockey', 'sci.med', 'sci.space', 'talk.politics.misc']

In [56]:
def findTFICF(d,wl,cw):
    tficf={}
    c=set(Counter(wl))
    for i in c:
        tf=c[i]
        icf=np.log(len(d)/cw+1)
        tficf[i]=tf*icf
    return tficf

In [60]:
def accuracy(yt,yp):
    c=0
    for i in range(len(yp)):
        if(yp[i]==yt[i]):
            c+=1
    return c/len(yp)

In [86]:
def confusion_matrix(yt,yp):
    cm= np.zeros((len(labels), len(labels))).astype(int)
    for i in range(len(yp)):
        cm[labels.index(yp[i])][labels.index(yt[i])]+= 1
    return cm

In [107]:
class NB:
    def fit(self,x_train,y_train,k):
        w=x_train
        cls=y_train
        n=len(w)
        d={}
        #dictonary of words per class
        for i in range(n):
            if cls[i] in d:
                d[cls[i]]=d[cls[i]]+w[i]
            else:
                d[cls[i]]=w[i]
        #list of words
        wl=[]
        for i in wl:
            wl=wl+w[i]
        #count of word per class
        cw={}
        for i in d:
            l=d[i]
            for j in set(l):
                if j not in cw:
                    cw[j]=1
                else:
                    cw[j]+=1
        #calculating TF-ICF
        tficf=findTFICF(d,wl,cw)
        x=sorted(tficf.items(), key=operator.itemgetter(1), reverse=True)
        x=x[:int(len(x)*k/100)]# considering top k features
        uw=[i[0] for i in x]
    
        #frquency of each word per class
        f={}
        #total number of words in that class
        nw={}
        for i in labels:
            fl=Counter(d[i])
            for j in uw:
                f[i,j]=fl[j]
                if i not in nw:
                    nw[i]=fl[i]
                else:
                    nw[i]+=fl[j]
        
        #calculating the frequency of each class 
        train={}
        for i in y_train:
            if i not in train:
                train[i]=1
            else:
                train[i]+=1
        self.f=f
        self.uw=uw
        self.nw=nw
                
    #calculating the probability of each word in test set and returning the classes for each set of words which have max probability acc to class
    def predict(self,x_test):
        yp=[]
        for i in range(len(x_test)):
            cwp=[]
            for l in labels:
                wp=0
                for w in x_test[i]:
                    try:
                        f=self.f[l,w]
                    except:
                        f=0
                    
                    c=self.nw[l]
                    p=(f+1)+(c+len(self.uw))
                    wp+=np.log(p)
                cwp.append(wp)
            yp.append(cls[np.argmax(cwp)])
        return yp

In [ ]:
ratio=[0.5,0.7,0.8]
for i in range(3):
  train=data.sample(frac=ratio[i],random_state=41)
  x_train,y_train=train['text'].tolist(),train['label'].tolist()
  test=data.sample(frac=1,random_state=41).drop(train.index)
  x_test,y_test=test['text'].tolist(),test['label'].tolist()
  nb=NB()
  nb.fit(X_train,y_train)
  yp=nb.predict(x_test)
  acc=nb.cal_accuracy(yp,y_test)*100
  print(f"Accuray at {ratio[i]*100}:{100-ratio[i]*100} fraction split is {acc}")
  print("Confusion Matrix is:\n")
  print(nb.confusion_matrix(yp,y_test))
  print("\n\n")